In [2]:
import pandas as pd
import math
import random
import numpy as np
import seaborn as sns
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC, NuSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [3]:
# training and testing data
train_data = pd.read_csv("/Users/samkamarfua/Downloads/online_jewellery_shop_train.csv")
test_data = pd.read_csv("/Users/samkamarfua/Downloads/online_jewellery_shop_test_final.csv")
print('done')

done


In [4]:
# converting VisitorType to categorical variate train data
train_data["VisitorType"] = train_data["VisitorType"].replace(['Returning_Visitor'], 0)
train_data["VisitorType"] = train_data["VisitorType"].replace(['New_Visitor'],1)
train_data["VisitorType"] = train_data["VisitorType"].replace(['Other'],3)
print('done')

done


In [5]:
# converting VisitorType to categorical variate test data
test_data["VisitorType"] = test_data["VisitorType"].replace(['Returning_Visitor'], 0)
test_data["VisitorType"] = test_data["VisitorType"].replace(['New_Visitor'],1)
test_data["VisitorType"] = test_data["VisitorType"].replace(['Other'],3)
print('done')

done


In [6]:
# converting Month to categorical variate for train_data
train_data["Month"] = train_data["Month"].replace(['Jan','Feb','Mar', 'Apr'],0) # Winter
train_data["Month"] = train_data["Month"].replace(['May', 'June', 'Jul', 'Aug'],1) # Spring
train_data["Month"] = train_data["Month"].replace(["Sep", "Oct", "Nov", "Dec"],2) # Fall
print('done')

done


In [13]:
# converting Month to categorical variate for test_data
test_data["Month"] = test_data["Month"].replace(['Jan','Feb','Mar', 'Apr'],0) # winter
test_data["Month"] = test_data["Month"].replace(['May', 'June', 'Jul', 'Aug'],1) # spring
test_data["Month"] = test_data["Month"].replace(["Sep", "Oct", "Nov", "Dec"],2) # Fall
print('done')
train_data.shape

done


(8630, 19)

In [8]:
# correlation matrix for the last row helps to decide which variates we can safely ignore due to low correlation value
train_data.corr()
# Variables that can be ignored with low values < 0.075
# Region
# TrafficType
# OperatingSystems
# Browser
# Weekend
# ID
# Informational_Duration

,ID,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
ID,1.000000,0.024600,0.016491,0.014070,0.003930,0.019175,0.005059,0.005860,-0.001776,-0.009265,-0.007109,0.013096,0.016516,-0.001628,0.014797,-0.006849,0.010090,0.029847,0.018259
Administrative,0.024600,1.000000,0.590137,0.376724,0.256741,0.415619,0.347251,-0.224929,-0.317254,0.102725,-0.099414,0.093284,-0.006934,-0.015023,-0.010988,-0.045683,0.013582,0.026793,0.128718
Administrative_Duration,0.016491,0.590137,1.000000,0.305479,0.257696,0.291914,0.370026,-0.139767,-0.198634,0.061145,-0.071025,0.055709,-0.012048,-0.015025,-0.011620,-0.019740,0.008367,0.017642,0.076243
Informational,0.014070,0.376724,0.305479,1.000000,0.620708,0.389791,0.405058,-0.113051,-0.158515,0.041260,-0.044203,0.062010,-0.007930,-0.025835,-0.035331,-0.034570,-0.055218,0.035356,0.082617
Informational_Duration,0.003930,0.256741,0.257696,0.620708,1.000000,0.310350,0.389792,-0.071077,-0.100182,0.023110,-0.034232,0.035272,-0.012146,-0.015741,-0.033685,-0.026534,-0.041716,0.019851,0.067491
ProductRelated,0.019175,0.415619,0.291914,0.389791,0.310350,1.000000,0.853600,-0.203630,-0.287992,0.046407,-0.025327,0.155369,0.000499,-0.008348,-0.035755,-0.051562,-0.127511,0.020810,0.153061
ProductRelated_Duration,0.005059,0.347251,0.370026,0.405058,0.389792,0.853600,1.000000,-0.179851,-0.241971,0.045825,-0.035582,0.128244,0.003281,-0.006124,-0.036007,-0.042783,-0.113902,0.013146,0.141329
BounceRates,0.005860,-0.224929,-0.139767,-0.113051,-0.071077,-0.203630,-0.179851,1.000000,0.914937,-0.119522,0.080316,-0.054674,0.020293,-0.012674,-0.007918,0.082253,-0.097205,-0.047162,-0.153888
ExitRates,-0.001776,-0.317254,-0.198634,-0.158515,-0.100182,-0.287992,-0.241971,0.914937,1.000000,-0.174251,0.108983,-0.084516,0.012984,-0.006467,-0.007060,0.081116,-0.130046,-0.065657,-0.207834
PageValues,-0.009265,0.102725,0.061145,0.041260,0.023110,0.046407,0.045825,-0.119522,-0.174251,1.000000,-0.058202,0.065108,0.015369,0.054407,0.017722,0.020527,0.133014,0.012268,0.497922


In [9]:
# checking the NaN value rows for train data
is_nan = train_data.isnull()
row_has_nan = is_nan.any(axis = 1)
train_data[row_has_nan]

,ID,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
502,3934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,2,2,1,2,0,0,0
1597,7032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,2,2,3,1,0,0,0
1974,5695,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,1,1,1,2,0,0,0
3801,8387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,3,2,1,1,0,0,0
3833,7446,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,2,4,5,1,0,0,0
4270,5563,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,2,2,1,1,0,1,0
6665,12293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,2,2,2,1,0,0,0
8252,10419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,2,2,1,2,0,0,0


In [10]:
# checking the NaN value rows for test data
is_nan = test_data.isnull()
row_has_nan = is_nan.any(axis = 1)
test_data[row_has_nan]

,ID,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend
145,1882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,3,2,4,1,0,0
1978,1378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,3,2,4,15,0,1
2231,2954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1,2,2,4,13,0,0
2461,1116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,2,2,1,2,0,0
3293,2741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,1,1,2,3,0,0
3328,1613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,1,1,6,1,0,1


In [11]:
# fill the nan values with 0
#train_data = train_data.dropna()
train_data = train_data.fillna(0)
test_data = test_data.fillna(0)


In [12]:
y_train_orig = xtrain_orig["Revenue"]
x_train_orig = xtrain_orig.drop(["Revenue", "ID","OperatingSystems" ], axis=1)
x_test_orig = test_data.drop(["ID", "OperatingSystems"], axis=1)

x_train_orig.shape

NameError: name 'xtrain_orig' is not defined

In [238]:
pipeline = make_pipeline(StandardScaler(), MLPClassifier(hidden_layer_sizes = (10,10,10), max_iter = 1500,
                                                        activation = 'logistic'))
scores = cross_val_score(pipeline, X = x_train_orig, y = y_train_orig, cv=10,n_jobs=-1, scoring = 'roc_auc')
print(scores)
print(sum(scores)/10)

[0.89276867 0.889196   0.91816637 0.9323035  0.93248777 0.87099598
 0.95035534 0.93137295 0.90493357 0.91643836]
0.9139018501472513


In [240]:
rfc = MLPClassifier(hidden_layer_sizes = (10,10,10), max_iter =1000, activation = 'logistic')
rfc.fit(x_train_orig, y_train_orig)
pred_test= rfc.predict(x_test_orig)
print('helo')

helo


In [241]:

print(sum(pred_test))

448


In [242]:
print(sum(pred_test))
rev = rfc.predict_proba(x_test_orig)
rev = rev[0:len(x_test_orig),1]

448
[0.02832619 0.72120914 0.02924843 ... 0.05400158 0.02834863 0.02851801]


In [219]:
ID = test_data["ID"]
d = {'ID': ID, 'Revenue':pd.Series(rev)}
df = pd.DataFrame(data = d)
x = df.to_csv("/Users/samkamarfua/Downloads/dc1_result_final_11.csv", index = False)